# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()


API key looks good so far


In [ ]:
# set up environment

In [3]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Get gpt-4o-mini to answer, with streaming

def stream_answer(question):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
             {"role": "system", "content": "You are a helpful assistant who explains Python code clearly in Polish."}
            {"role": "user", "content": question},
    
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [7]:
stream_answer(question)

Certainly! Let's break down the code snippet:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

### What does this do?

1. **Set Comprehension:**

```python
{book.get("author") for book in books if book.get("author")}
```

- This creates a **set** containing all the unique author names found in the `books` iterable.
- For each `book` in `books`, it:
  - Calls `book.get("author")` to retrieve the author name.
  - The `if book.get("author")` condition ensures only books that have a non-None, truthy "author" value are included.
- Using a set comprehension means duplicates are removed because sets only store unique values.

2. **`yield from`**:

- `yield from` is used in a generator function to **yield all values from the given iterable one by one**.
- Here, the iterable is the set of unique authors constructed above.
- So this statement will yield each unique author name one at a time from the generator.

### Why write it this way?

- Using a set comprehension filters out duplicates and ignores any books without authors.
- `yield from` simplifies the code as opposed to writing an explicit loop:

```python
authors = {book.get("author") for book in books if book.get("author")}
for author in authors:
    yield author
```

- This makes it concise and Pythonic.
- It allows the enclosing function to be a generator that yields unique authors in no particular order (since sets are unordered).

### Summary:

- This line is typically inside a generator function.
- It yields unique author names from a collection of book dictionaries.
- It skips any book that has no author specified.
- The use of `yield from` makes code cleaner by directly delegating iteration over the set of authors.

---

If you want, I can also provide an example usage or how this fits into a function.

In [ ]:
# Get Llama 3.2 to answer